In [1]:
import pandas as pd
from  sklearn.linear_model import SGDClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
from lightgbm import LGBMClassifier

In [3]:
from sklearn.model_selection import KFold

### Загружаем данные в виде pandas.DataFrame и делим на test и train

In [4]:
df = pd.read_csv('result2.csv', delimiter=',')
df.shape

(385, 157)

In [32]:
features_target = features + ['target']

train = df[:300][features_target]
test  = df[300:][features_target]

In [33]:
train.shape

(300, 11)

In [8]:
bad = []
# Проверяем на наличие NULL-ов
for a in test:
    if test[a].isnull().sum() != 0:
        print(a, test[a].isnull().sum())
        bad.append(a)

moments_normalized-0-0 63
moments_normalized-0-1 63
moments_normalized-1-0 63
weighted_moments_normalized-0-0 63
weighted_moments_normalized-0-1 63
weighted_moments_normalized-1-0 63


In [4]:
# Чтоб не париться заполняем NULL-ы нулями
train = train.fillna(0)
test = test.fillna(0)

In [31]:
train

,bbox_area,convex_area,eccentricity,equivalent_diameter,filled_area,major_axis_length,minor_axis_length,perimeter,perimeter_crofton,solidity,target
65,484,358,0.166481,16.195288,342,24.490923,24.149145,116.468037,110.418623,0.575419,круг
66,1666,1479,0.723981,26.916042,1342,63.723881,43.958021,286.628463,273.277931,0.384719,прямоугольник
67,1085,691,0.473875,22.482796,397,37.532259,33.050620,169.338095,163.224098,0.574530,треугольник
68,1296,1019,0.244446,20.153496,993,46.646920,45.231787,216.208153,206.318941,0.313052,круг
69,3477,2211,0.471600,31.655008,2142,72.403461,63.846278,348.918831,330.795795,0.355948,квадрат
...,...,...,...,...,...,...,...,...,...,...,...
380,784,481,0.495325,17.517155,481,33.048315,28.709327,142.308658,134.917068,0.501040,квадрат
381,5928,4147,0.646612,43.482879,3490,92.609440,70.644275,686.465079,644.105912,0.358090,mixed
382,864,513,0.835926,19.248707,481,41.819450,22.952278,160.852814,152.498030,0.567251,треугольник
383,1806,889,0.859097,20.621882,871,59.760609,30.586245,225.989899,215.592617,0.375703,прямоугольник


In [13]:
test.to_csv("example_predict.csv")

### Создаём и обучаем простые модели

In [12]:
features = ['bbox_area', 'convex_area', 'eccentricity', 'equivalent_diameter', 'filled_area', 'major_axis_length', 'minor_axis_length','perimeter','perimeter_crofton','solidity']

In [42]:
# KNN Classifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [9]:
# DecisionTree
clf = DecisionTreeClassifier()
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [12]:
# Random Forest
clf = RandomForestClassifier(n_estimators = 10)
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [42]:
# Boosting
model = LGBMClassifier(n_estimators = 500, max_depth = 3)
model.fit(train[features], train['target'])

predict = model.predict(test[features])

In [14]:
test.loc[i, 'predict'] = predict[i]

<ipython-input-14-fd4577cbb27c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predict'] = predict


In [16]:
# Считаю процент верных предположений
def accuracy(test):
    i = 0
    for index, row in test.iterrows():
        if row['target'] == row['predict']:
            i = i+1

    perc = i/test.shape[0]
    return perc

per = accuracy(test)
print(per)

0.7076923076923077


In [39]:
# Считаю процент верных предположений

def acc(y_test, predict):
    i = 0
    for k in range(len(y_test)):
        if y_test[k] == predict[k]:
            i = i+1
    perc = i/len(y_test)
    return perc

#per = acc(y_test, predict)
#print(per)

### Кроссвалидация

In [35]:
kf = KFold(n_splits=4)
kf.get_n_splits(train)

4

In [40]:
model = LGBMClassifier(n_estimators = 500, max_depth = 3)

acc_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, features], train.loc[test_index, features ]
    y_train, y_test = train.loc[train_index, 'target'], train.loc[test_index, 'target']

    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    accur = acc(list(y_test), list(predict))
    acc_list.append(accur)
    print(i, accur)
    
print("mean rmse for 5-fold: {}".format(str(np.mean(acc_list))))
print("std roc_auc for 5-fold: {}".format(str(np.std(acc_list))))

0 0.72
1 0.72
2 0.8133333333333334
3 0.7333333333333333


NameError: name 'np' is not defined

In [41]:
from sklearn.inspection import permutation_importance

In [53]:
result = permutation_importance(model, test[features], test['target'], n_repeats=5, random_state=0)

In [59]:
i = 0
for r in result['importances_mean']:
    print(features[i], r)
    i = i+1

bbox_area 0.08
convex_area 0.14117647058823532
eccentricity 0.15764705882352942
equivalent_diameter 0.08235294117647059
filled_area 2.2204460492503132e-17
major_axis_length 0.00235294117647058
minor_axis_length -0.011764705882352922
perimeter 0.08470588235294119
perimeter_crofton 0.040000000000000015
solidity 0.08705882352941177
